## Projeto para Disciplina de Mineração de Dados  
## Aplicação de Técnica de classificação para Solução de Captcha  Sonoro
### Programa de Pós Graduação em Computação UFABC - 2º Quadrimestre de 2018



#### Integrantes:
Erick Fernandes da Cruz <br>
Gustavo Borges Lugoboni <br>
Igor Esteves de Oliveira 

## Código Desenvolvido

A seguir se encontram as bibliotecas que foram utilizadas para o desenvolvimento do presente projeto necessárias

In [50]:
#Bibliotecas utilizadas

import os
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import IPython.display as ipd
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import tree

from bokeh.models import CDSView, ColumnDataSource, GroupFilter
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

A primeira etapa que desenvolvemos foi a separação das letras que eram obtidas em cada audio que continha 4 caracteres, para isso utilizamos 2 arrays inicialmente, um deles contendo a onda especifica de cada letra do conjunto de treinamento e o outro com a letra correspondente

In [51]:
ArrayLetras = list()
Y = list()

Para separar as letras desenvolvemos a função separar_letras que tem como objetivo receber um audio com 4 letras, dividir em 4 audios menores e a partir desses audios menores eliminar a maior parte do ruído possível através de uma frequência de corte para ter uma melhor representação de cada letra

In [52]:
def separar_letras(data, fs, destination_array) :

    #Separar em 4 conjuntos 
    n = 4
    splited = []
    len_l = len(data)
    for i in range(n):
        start = int(i*len_l/n)
        end = int((i+1)*len_l/n)
        splited.append(data[start:end])
    
    frame_lenght = 6300 # frames de fs/14 aprox 0.5s
    
    chars_values = list()
    
    for audioPart in range(4):
        initial_frame_pos = 0 
        frames_list = list()

        #Separando os frames
        while initial_frame_pos < len(splited[audioPart]):
            frames_list.append(splited[audioPart][initial_frame_pos : initial_frame_pos+frame_lenght])
            initial_frame_pos += frame_lenght

        #Filtrando por amplitude maxima do frame inicial (supondo que é sempre ruido ou silencio) + desvio padrao
        amp_cat_values = list()

        max_values = [max(x) for x in frames_list] 
        max_values.sort()
        max_value_mean = np.mean(max_values)
        max_value_std = np.std(max_values)

        for frame in frames_list:
            frame_max = max(abs(frame))
            #se o frame tiver amplitude maxima <= a inicial std consideramos como um trecho de audio valido      
            if frame_max <= max_value_mean + max_value_std**2 :  
                if frame_max >= max_value_mean - max_value_std**2 :
                    amp_cat_values.append("max")
                else:
                    amp_cat_values.append("min")
            else:
                amp_cat_values.append("max")

        # guardando os idices dos frames de frames_list onde a amplitude equivale a um valor de audio valido
        max_indexes = list()
        index_list = range(0, len(amp_cat_values))
        amp_cat_values = list(zip(index_list, amp_cat_values))

        max_indexes = [index for index in amp_cat_values if index[1] is "max"]

        #agrupando por indices a uma distancia <= 2 posições (separando os caracteres)
        chars_indexes = list()
        new_frame = list()
        new_frame.append(max_indexes[0])
        for frame_info in max_indexes:
            if frame_info[0] - new_frame[-1][0] <= 2:
                if frame_info not in new_frame:
                    new_frame.append(frame_info)
            else:
                chars_indexes.append(new_frame)
                new_frame = list()
                new_frame.append(frame_info)
            if frame_info is max_indexes[-1]:
                chars_indexes.append(new_frame)

        # filtrando os indices no vetor de frames_list (contendo as amostras originais) para separar as amostras de
        # caracteres validos
        initialBigger = 0
        lastBigger = 0

        diferenca = 0

        for index in chars_indexes:
            initial = index[0][0] 
            last = index[-1][0] 

            if (last - initial) > diferenca:
                initialBigger = initial
                lastBigger = last

        if (initialBigger != 0):
            initialBigger = initialBigger - 1
        
        value = np.hstack(frames_list[initialBigger:lastBigger+1])
        destination_array.append(value)

Agora que tinhamos a função pronta criamos um laço que percorre todos os arquivos na pasta do conjunto de treinamento para que pudessemos recolher os dados para que estes fossem manipulados em um momento futuro

In [53]:
for root, dirs, files in os.walk("treino/"):  
    
    for filename in files:
        
        data, fs = librosa.load('treino/' + filename, None)
        for s in range(4):
            Y.append(filename[s])
            
        separar_letras(data, fs, ArrayLetras)

print(len(ArrayLetras))


800


# A PARTIR DAQUI É TUDO TESTE

In [85]:
#extraindo informações
duration = np.array([len(x)/fs for x in ArrayLetras])

tempo_beat = [librosa.beat.beat_track(y=x, sr=fs) for x in ArrayLetras]
tempo = np.array([x[0] for x in tempo_beat])
beat_frame = np.array([x[1] for x in tempo_beat])

mean_percussive = np.array([librosa.effects.hpss(x)[1].mean() for x in ArrayLetras])

#sftf = [librosa.stft(x) for x in ArrayLetras]
fourier = [np.fft.fft(x) for x in ArrayLetras]
mean_freq = np.array([np.abs(x).mean() for x in fourier])

mean_centroid = [librosa.feature.spectral_centroid(y=x, sr=fs).mean() for x in ArrayLetras]

freq_QT50 = [np.percentile(np.abs(x), 50) for x in sftf]
freq_QT75 = [np.percentile(np.abs(x), 75) for x in sftf]
freq_QT25 = [np.percentile(np.abs(x), 25) for x in sftf]


train_base = np.array(list(zip(
    duration, 
    tempo, 
    mean_percussive, 
    mean_freq, 
    mean_centroid, 
    freq_QT50, 
    freq_QT75, 
    freq_QT25)))


In [86]:
#carregando base de testes
Array_test = list()
Y_test = list()
for root, dirs, files in os.walk("teste/"):  
    
    for filename in files:
        
        data, fs = librosa.load('teste/' + filename, None)
        for s in range(4):
            Y_test.append(filename[s])
            
        separar_letras(data, fs, Array_test)


In [87]:
#extraindo informações
duration_test = np.array([len(x)/fs for x in Array_test])

tempo_beat_test = [librosa.beat.beat_track(y=x, sr=fs) for x in Array_test]
tempo_test = np.array([x[0] for x in tempo_beat_test])
beat_frame_test = np.array([x[1] for x in tempo_beat_test])

mean_percussive_test = np.array([librosa.effects.hpss(x)[1].mean() for x in Array_test])

#sftf_test = [librosa.stft(x) for x in Array_test]
fourier_test = [np.fft.fft(x) for x in ArrayLetras]
mean_freq_test = np.array([np.abs(x).mean() for x in fourier_test])
freq_QT50_test = [np.percentile(np.abs(x), 50) for x in sftf]
freq_QT75_test = [np.percentile(np.abs(x), 75) for x in sftf]
freq_QT25_test = [np.percentile(np.abs(x), 25) for x in sftf]

mean_centroid_test = [librosa.feature.spectral_centroid(y=x, sr=fs).mean() for x in Array_test]



test_base = np.array(list(zip(
    duration_test, 
    tempo_test, 
    mean_percussive_test, 
    mean_freq_test, 
    mean_centroid_test, 
    freq_QT50_test, 
    freq_QT75_test, 
    freq_QT25_test)))

In [88]:
#Criando bases de cada letra a,b,c,d,h,m,n,x,6,7

a_base = list()
b_base = list()
c_base = list()
d_base = list()
h_base = list()
m_base = list()
n_base = list()
x_base = list()
si_base = list()
se_base = list()
labels = ['duration', 
          'tempo', 
          'mean_percussive', 
          'mean_freq', 
          'mean_centroid', 
          'freq_QT50', 
          'freq_QT75', 
          'freq_QT25']

for i in range(0, len(Y)):
    if Y[i] == 'a':
        a_base.append(train_base[i])
    elif Y[i] == 'b':
        b_base.append(train_base[i])
    elif Y[i] == 'c':
        c_base.append(train_base[i])
    elif Y[i] == 'd':
        d_base.append(train_base[i])
    elif Y[i] == 'h':
        h_base.append(train_base[i])
    elif Y[i] == 'm':
        m_base.append(train_base[i])
    elif Y[i] == 'n':
        n_base.append(train_base[i])
    elif Y[i] == 'x':
        x_base.append(train_base[i])
    elif Y[i] == '6':
        si_base.append(train_base[i])
    elif Y[i] == '7':
        se_base.append(train_base[i])
    else:
        pass

a_base = pd.DataFrame.from_records(a_base, columns=labels)
b_base = pd.DataFrame.from_records(b_base, columns=labels)
c_base = pd.DataFrame.from_records(c_base, columns=labels)
d_base = pd.DataFrame.from_records(d_base, columns=labels)
h_base = pd.DataFrame.from_records(h_base, columns=labels)
m_base = pd.DataFrame.from_records(m_base, columns=labels)
n_base = pd.DataFrame.from_records(n_base, columns=labels)
x_base = pd.DataFrame.from_records(x_base, columns=labels)
si_base = pd.DataFrame.from_records(si_base, columns=labels)
se_base = pd.DataFrame.from_records(se_base, columns=labels)


#Base de teste

a_base_test = list()
b_base_test = list()
c_base_test = list()
d_base_test = list()
h_base_test = list()
m_base_test = list()
n_base_test = list()
x_base_test = list()
si_base_test = list()
se_base_test = list()

for i in range(0, len(Y_test)):
    if Y[i] == 'a':
        a_base_test.append(test_base[i])
    elif Y[i] == 'b':
        b_base_test.append(test_base[i])
    elif Y[i] == 'c':
        c_base_test.append(test_base[i])
    elif Y[i] == 'd':
        d_base_test.append(test_base[i])
    elif Y[i] == 'h':
        h_base_test.append(test_base[i])
    elif Y[i] == 'm':
        m_base_test.append(test_base[i])
    elif Y[i] == 'n':
        n_base_test.append(test_base[i])
    elif Y[i] == 'x':
        x_base_test.append(test_base[i])
    elif Y[i] == '6':
        si_base_test.append(test_base[i])
    elif Y[i] == '7':
        se_base_test.append(test_base[i])
    else:
        pass

a_base_test = pd.DataFrame.from_records(a_base_test, columns=labels)
b_base_test = pd.DataFrame.from_records(b_base_test, columns=labels)
c_base_test = pd.DataFrame.from_records(c_base_test, columns=labels)
d_base_test = pd.DataFrame.from_records(d_base_test, columns=labels)
h_base_test = pd.DataFrame.from_records(h_base_test, columns=labels)
m_base_test = pd.DataFrame.from_records(m_base_test, columns=labels)
n_base_test = pd.DataFrame.from_records(n_base_test, columns=labels)
x_base_test = pd.DataFrame.from_records(x_base_test, columns=labels)
si_base_test = pd.DataFrame.from_records(si_base_test, columns=labels)
se_base_test = pd.DataFrame.from_records(se_base_test, columns=labels)

<h2>Análise Exploratória</h2>

In [89]:
output_notebook()
#histogramas
#mean_freq
a_mean_freq_hist, edges_a = np.histogram(a_base.mean_freq, density=True)
b_mean_freq_hist, edges_b = np.histogram(b_base.mean_freq, density=True)
c_mean_freq_hist, edges_c = np.histogram(c_base.mean_freq, density=True)
d_mean_freq_hist, edges_d = np.histogram(d_base.mean_freq, density=True)
h_mean_freq_hist, edges_h = np.histogram(h_base.mean_freq, density=True)
m_mean_freq_hist, edges_m = np.histogram(m_base.mean_freq, density=True)
n_mean_freq_hist, edges_n = np.histogram(n_base.mean_freq, density=True)
x_mean_freq_hist, edges_x = np.histogram(x_base.mean_freq, density=True)
si_mean_freq_hist, edges_si = np.histogram(si_base.mean_freq, density=True)
se_mean_freq_hist, edges_se = np.histogram(se_base.mean_freq, density=True)

fig_a_mean_freq_hist = figure(title="a_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_a_mean_freq_hist.quad(top=a_mean_freq_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_mean_freq_hist = figure(title="b_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_b_mean_freq_hist.quad(top=b_mean_freq_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_mean_freq_hist = figure(title="c_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_c_mean_freq_hist.quad(top=c_mean_freq_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_mean_freq_hist = figure(title="d_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_d_mean_freq_hist.quad(top=d_mean_freq_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_mean_freq_hist = figure(title="h_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_h_mean_freq_hist.quad(top=h_mean_freq_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_mean_freq_hist = figure(title="m_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_m_mean_freq_hist.quad(top=m_mean_freq_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_mean_freq_hist = figure(title="n_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_n_mean_freq_hist.quad(top=n_mean_freq_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_mean_freq_hist = figure(title="x_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_x_mean_freq_hist.quad(top=x_mean_freq_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_mean_freq_hist = figure(title="six_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_si_mean_freq_hist.quad(top=si_mean_freq_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_mean_freq_hist = figure(title="seven_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_se_mean_freq_hist.quad(top=se_mean_freq_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_mean_freq_hist, fig_b_mean_freq_hist, fig_c_mean_freq_hist], 
                [fig_d_mean_freq_hist, fig_h_mean_freq_hist, fig_m_mean_freq_hist], 
                [fig_n_mean_freq_hist, fig_x_mean_freq_hist, fig_si_mean_freq_hist], 
                [fig_se_mean_freq_hist]], title='MEAN_FREQ', notebook_handle=True)
show(plt)

Loading BokehJS ...

In [90]:
#freq_QT50 histograma
a_freq_QT50_hist, edges_a = np.histogram(a_base.freq_QT50, density=True)
b_freq_QT50_hist, edges_b = np.histogram(b_base.freq_QT50, density=True)
c_freq_QT50_hist, edges_c = np.histogram(c_base.freq_QT50, density=True)
d_freq_QT50_hist, edges_d = np.histogram(d_base.freq_QT50, density=True)
h_freq_QT50_hist, edges_h = np.histogram(h_base.freq_QT50, density=True)
m_freq_QT50_hist, edges_m = np.histogram(m_base.freq_QT50, density=True)
n_freq_QT50_hist, edges_n = np.histogram(n_base.freq_QT50, density=True)
x_freq_QT50_hist, edges_x = np.histogram(x_base.freq_QT50, density=True)
si_freq_QT50_hist, edges_si = np.histogram(si_base.freq_QT50, density=True)
se_freq_QT50_hist, edges_se = np.histogram(se_base.freq_QT50, density=True)

fig_a_freq_QT50_hist = figure(title="a_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_a_freq_QT50_hist.quad(top=a_freq_QT50_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_freq_QT50_hist = figure(title="b_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_b_freq_QT50_hist.quad(top=b_freq_QT50_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_freq_QT50_hist = figure(title="c_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_c_freq_QT50_hist.quad(top=c_freq_QT50_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_freq_QT50_hist = figure(title="d_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_d_freq_QT50_hist.quad(top=d_freq_QT50_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_freq_QT50_hist = figure(title="h_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_h_freq_QT50_hist.quad(top=h_freq_QT50_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_freq_QT50_hist = figure(title="m_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_m_freq_QT50_hist.quad(top=m_freq_QT50_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_freq_QT50_hist = figure(title="n_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_n_freq_QT50_hist.quad(top=n_freq_QT50_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_freq_QT50_hist = figure(title="x_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_x_freq_QT50_hist.quad(top=x_freq_QT50_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_freq_QT50_hist = figure(title="six_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_si_freq_QT50_hist.quad(top=si_freq_QT50_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_freq_QT50_hist = figure(title="seven_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_se_freq_QT50_hist.quad(top=se_freq_QT50_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_freq_QT50_hist, fig_b_freq_QT50_hist, fig_c_freq_QT50_hist], 
                [fig_d_freq_QT50_hist, fig_h_freq_QT50_hist, fig_m_freq_QT50_hist], 
                [fig_n_freq_QT50_hist, fig_x_freq_QT50_hist, fig_si_freq_QT50_hist], 
                [fig_se_freq_QT50_hist]], title='freq_QT50', notebook_handle=True)
show(plt)

In [99]:
#freq_QT25 histograma
a_freq_QT25_hist, edges_a = np.histogram(a_base.freq_QT25, density=True)
b_freq_QT25_hist, edges_b = np.histogram(b_base.freq_QT25, density=True)
c_freq_QT25_hist, edges_c = np.histogram(c_base.freq_QT25, density=True)
d_freq_QT25_hist, edges_d = np.histogram(d_base.freq_QT25, density=True)
h_freq_QT25_hist, edges_h = np.histogram(h_base.freq_QT25, density=True)
m_freq_QT25_hist, edges_m = np.histogram(m_base.freq_QT25, density=True)
n_freq_QT25_hist, edges_n = np.histogram(n_base.freq_QT25, density=True)
x_freq_QT25_hist, edges_x = np.histogram(x_base.freq_QT25, density=True)
si_freq_QT25_hist, edges_si = np.histogram(si_base.freq_QT25, density=True)
se_freq_QT25_hist, edges_se = np.histogram(se_base.freq_QT25, density=True)

fig_a_freq_QT25_hist = figure(title="a_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_a_freq_QT25_hist.quad(top=a_freq_QT25_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_freq_QT25_hist = figure(title="b_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_b_freq_QT25_hist.quad(top=b_freq_QT25_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_freq_QT25_hist = figure(title="c_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_c_freq_QT25_hist.quad(top=c_freq_QT25_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_freq_QT25_hist = figure(title="d_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_d_freq_QT25_hist.quad(top=d_freq_QT25_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_freq_QT25_hist = figure(title="h_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_h_freq_QT25_hist.quad(top=h_freq_QT25_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_freq_QT25_hist = figure(title="m_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_m_freq_QT25_hist.quad(top=m_freq_QT25_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_freq_QT25_hist = figure(title="n_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_n_freq_QT25_hist.quad(top=n_freq_QT25_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_freq_QT25_hist = figure(title="x_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_x_freq_QT25_hist.quad(top=x_freq_QT25_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_freq_QT25_hist = figure(title="six_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_si_freq_QT25_hist.quad(top=si_freq_QT25_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_freq_QT25_hist = figure(title="seven_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_se_freq_QT25_hist.quad(top=se_freq_QT25_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_freq_QT25_hist, fig_b_freq_QT25_hist, fig_c_freq_QT25_hist], 
                [fig_d_freq_QT25_hist, fig_h_freq_QT25_hist, fig_m_freq_QT25_hist], 
                [fig_n_freq_QT25_hist, fig_x_freq_QT25_hist, fig_si_freq_QT25_hist], 
                [fig_se_freq_QT25_hist]], title='freq_QT25', notebook_handle=True)
show(plt)

In [98]:
#freq_QT75 histograma
a_freq_QT75_hist, edges_a = np.histogram(a_base.freq_QT75, density=True)
b_freq_QT75_hist, edges_b = np.histogram(b_base.freq_QT75, density=True)
c_freq_QT75_hist, edges_c = np.histogram(c_base.freq_QT75, density=True)
d_freq_QT75_hist, edges_d = np.histogram(d_base.freq_QT75, density=True)
h_freq_QT75_hist, edges_h = np.histogram(h_base.freq_QT75, density=True)
m_freq_QT75_hist, edges_m = np.histogram(m_base.freq_QT75, density=True)
n_freq_QT75_hist, edges_n = np.histogram(n_base.freq_QT75, density=True)
x_freq_QT75_hist, edges_x = np.histogram(x_base.freq_QT75, density=True)
si_freq_QT75_hist, edges_si = np.histogram(si_base.freq_QT75, density=True)
se_freq_QT75_hist, edges_se = np.histogram(se_base.freq_QT75, density=True)

fig_a_freq_QT75_hist = figure(title="a_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_a_freq_QT75_hist.quad(top=a_freq_QT75_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_freq_QT75_hist = figure(title="b_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_b_freq_QT75_hist.quad(top=b_freq_QT75_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_freq_QT75_hist = figure(title="c_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_c_freq_QT75_hist.quad(top=c_freq_QT75_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_freq_QT75_hist = figure(title="d_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_d_freq_QT75_hist.quad(top=d_freq_QT75_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_freq_QT75_hist = figure(title="h_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_h_freq_QT75_hist.quad(top=h_freq_QT75_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_freq_QT75_hist = figure(title="m_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_m_freq_QT75_hist.quad(top=m_freq_QT75_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_freq_QT75_hist = figure(title="n_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_n_freq_QT75_hist.quad(top=n_freq_QT75_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_freq_QT75_hist = figure(title="x_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_x_freq_QT75_hist.quad(top=x_freq_QT75_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_freq_QT75_hist = figure(title="six_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_si_freq_QT75_hist.quad(top=si_freq_QT75_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_freq_QT75_hist = figure(title="seven_freq_density", x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
fig_se_freq_QT75_hist.quad(top=se_freq_QT75_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_freq_QT75_hist, fig_b_freq_QT75_hist, fig_c_freq_QT75_hist], 
                [fig_d_freq_QT75_hist, fig_h_freq_QT75_hist, fig_m_freq_QT75_hist], 
                [fig_n_freq_QT75_hist, fig_x_freq_QT75_hist, fig_si_freq_QT75_hist], 
                [fig_se_freq_QT75_hist]], title='freq_QT75', notebook_handle=True)
show(plt)

In [91]:
#Tempo histogramas

a_tempo_hist, edges_a = np.histogram(a_base.tempo, density=True)
b_tempo_hist, edges_b = np.histogram(b_base.tempo, density=True)
c_tempo_hist, edges_c = np.histogram(c_base.tempo, density=True)
d_tempo_hist, edges_d = np.histogram(d_base.tempo, density=True)
h_tempo_hist, edges_h = np.histogram(h_base.tempo, density=True)
m_tempo_hist, edges_m = np.histogram(m_base.tempo, density=True)
n_tempo_hist, edges_n = np.histogram(n_base.tempo, density=True)
x_tempo_hist, edges_x = np.histogram(x_base.tempo, density=True)
si_tempo_hist, edges_si = np.histogram(si_base.tempo, density=True)
se_tempo_hist, edges_se = np.histogram(se_base.tempo, density=True)

fig_a_tempo_hist = figure(title="a_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_a_tempo_hist.quad(top=a_tempo_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_tempo_hist = figure(title="b_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_b_tempo_hist.quad(top=b_tempo_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_tempo_hist = figure(title="c_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_c_tempo_hist.quad(top=c_tempo_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_tempo_hist = figure(title="d_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_d_tempo_hist.quad(top=d_tempo_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_tempo_hist = figure(title="h_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_h_tempo_hist.quad(top=h_tempo_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_tempo_hist = figure(title="m_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_m_tempo_hist.quad(top=m_tempo_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_tempo_hist = figure(title="n_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_n_tempo_hist.quad(top=n_tempo_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_tempo_hist = figure(title="x_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_x_tempo_hist.quad(top=x_tempo_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_tempo_hist = figure(title="six_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_si_tempo_hist.quad(top=si_tempo_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_tempo_hist = figure(title="seven_tempo", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_se_tempo_hist.quad(top=se_tempo_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_tempo_hist, fig_b_tempo_hist, fig_c_tempo_hist], 
                [fig_d_tempo_hist, fig_h_tempo_hist, fig_m_tempo_hist], 
                [fig_n_tempo_hist, fig_x_tempo_hist, fig_si_tempo_hist], 
                [fig_se_tempo_hist]], title='tempo', notebook_handle=True)
show(plt)

In [79]:
#mean_percussive histograms 
a_mean_percussive_hist, edges_a = np.histogram(a_base.mean_percussive, density=True)
b_mean_percussive_hist, edges_b = np.histogram(b_base.mean_percussive, density=True)
c_mean_percussive_hist, edges_c = np.histogram(c_base.mean_percussive, density=True)
d_mean_percussive_hist, edges_d = np.histogram(d_base.mean_percussive, density=True)
h_mean_percussive_hist, edges_h = np.histogram(h_base.mean_percussive, density=True)
m_mean_percussive_hist, edges_m = np.histogram(m_base.mean_percussive, density=True)
n_mean_percussive_hist, edges_n = np.histogram(n_base.mean_percussive, density=True)
x_mean_percussive_hist, edges_x = np.histogram(x_base.mean_percussive, density=True)
si_mean_percussive_hist, edges_si = np.histogram(si_base.mean_percussive, density=True)
se_mean_percussive_hist, edges_se = np.histogram(se_base.mean_percussive, density=True)

fig_a_mean_percussive_hist = figure(title="a_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_a_mean_percussive_hist.quad(top=a_mean_percussive_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_b_mean_percussive_hist = figure(title="b_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_b_mean_percussive_hist.quad(top=b_mean_percussive_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_c_mean_percussive_hist = figure(title="c_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_c_mean_percussive_hist.quad(top=c_mean_percussive_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_d_mean_percussive_hist = figure(title="d_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_d_mean_percussive_hist.quad(top=d_mean_percussive_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_h_mean_percussive_hist = figure(title="h_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_h_mean_percussive_hist.quad(top=h_mean_percussive_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_m_mean_percussive_hist = figure(title="m_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_m_mean_percussive_hist.quad(top=m_mean_percussive_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_n_mean_percussive_hist = figure(title="n_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_n_mean_percussive_hist.quad(top=n_mean_percussive_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_x_mean_percussive_hist = figure(title="x_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_x_mean_percussive_hist.quad(top=x_mean_percussive_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_si_mean_percussive_hist = figure(title="six_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_si_mean_percussive_hist.quad(top=si_mean_percussive_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

fig_se_mean_percussive_hist = figure(title="seven_mean_percussive", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_se_mean_percussive_hist.quad(top=se_mean_percussive_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_mean_percussive_hist, fig_b_mean_percussive_hist, fig_c_mean_percussive_hist], 
                [fig_d_mean_percussive_hist, fig_h_mean_percussive_hist, fig_m_mean_percussive_hist], 
                [fig_n_mean_percussive_hist, fig_x_mean_percussive_hist, fig_si_mean_percussive_hist], 
                [fig_se_mean_percussive_hist]], title='mean_percussive', notebook_handle=True)
show(plt)

In [80]:
#Mean-centroid histograma

a_mean_centroid_hist, edges_a = np.histogram(a_base.mean_centroid, density=True)
b_mean_centroid_hist, edges_b = np.histogram(b_base.mean_centroid, density=True)
c_mean_centroid_hist, edges_c = np.histogram(c_base.mean_centroid, density=True)
d_mean_centroid_hist, edges_d = np.histogram(d_base.mean_centroid, density=True)
h_mean_centroid_hist, edges_h = np.histogram(h_base.mean_centroid, density=True)
m_mean_centroid_hist, edges_m = np.histogram(m_base.mean_centroid, density=True)
n_mean_centroid_hist, edges_n = np.histogram(n_base.mean_centroid, density=True)
x_mean_centroid_hist, edges_x = np.histogram(x_base.mean_centroid, density=True)
si_mean_centroid_hist, edges_si = np.histogram(si_base.mean_centroid, density=True)
se_mean_centroid_hist, edges_se = np.histogram(se_base.mean_centroid, density=True)

fig_a_mean_centroid_hist = figure(title="a_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_a_mean_centroid_hist.quad(top=a_mean_centroid_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_b_mean_centroid_hist = figure(title="b_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_b_mean_centroid_hist.quad(top=b_mean_centroid_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_c_mean_centroid_hist = figure(title="c_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_c_mean_centroid_hist.quad(top=c_mean_centroid_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_d_mean_centroid_hist = figure(title="d_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_d_mean_centroid_hist.quad(top=d_mean_centroid_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_h_mean_centroid_hist = figure(title="h_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_h_mean_centroid_hist.quad(top=h_mean_centroid_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_m_mean_centroid_hist = figure(title="m_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_m_mean_centroid_hist.quad(top=m_mean_centroid_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_n_mean_centroid_hist = figure(title="n_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_n_mean_centroid_hist.quad(top=n_mean_centroid_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_x_mean_centroid_hist = figure(title="x_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_x_mean_centroid_hist.quad(top=x_mean_centroid_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_si_mean_centroid_hist = figure(title="six_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_si_mean_centroid_hist.quad(top=si_mean_centroid_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

fig_se_mean_centroid_hist = figure(title="seven_mean_centroid", x_axis_label='value', y_axis_label='freq', plot_width=250, plot_height=250)
fig_se_mean_centroid_hist.quad(top=se_mean_centroid_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#033649", line_color="#033649", fill_alpha=0.3)

plt = gridplot([[fig_a_mean_centroid_hist, fig_b_mean_centroid_hist, fig_c_mean_centroid_hist], 
                [fig_d_mean_centroid_hist, fig_h_mean_centroid_hist, fig_m_mean_centroid_hist], 
                [fig_n_mean_centroid_hist, fig_x_mean_centroid_hist, fig_si_mean_centroid_hist], 
                [fig_se_mean_centroid_hist]], title='mean_centroid', notebook_handle=True)
show(plt)

In [104]:
#Cruzamento atributos
#mean_freq X tempo
fig_mean_freq_tempo = figure(x_axis_label='mean_freq', y_axis_label='tempo', title="Densidade Mean_freq x Tempo")
fig_mean_freq_tempo.circle(x=a_base.mean_freq, y=a_base.tempo, color='#ffe6ee', legend='a')
fig_mean_freq_tempo.circle(x=b_base.mean_freq, y=b_base.tempo, color='#b5e5fb', legend='b')
fig_mean_freq_tempo.circle(x=c_base.mean_freq, y=c_base.tempo, color='#000000', legend='c')
fig_mean_freq_tempo.circle(x=d_base.mean_freq, y=d_base.tempo, color='#5f37ab', legend='d')
fig_mean_freq_tempo.circle(x=h_base.mean_freq, y=h_base.tempo, color='#298b95', legend='h')
fig_mean_freq_tempo.circle(x=m_base.mean_freq, y=m_base.tempo, color='#561c22', legend='m')
fig_mean_freq_tempo.circle(x=n_base.mean_freq, y=n_base.tempo, color='#610361', legend='n')
fig_mean_freq_tempo.circle(x=x_base.mean_freq, y=x_base.tempo, color='#d82aac', legend='x')
fig_mean_freq_tempo.circle(x=si_base.mean_freq, y=si_base.tempo, color='#bc8b79', legend='si')
fig_mean_freq_tempo.circle(x=se_base.mean_freq, y=se_base.tempo, color='#6c819e', legend='se')


show(fig_mean_freq_tempo)

In [105]:
#mean_freq X mean_percussive

fig_mean_freq_mean_percussive = figure(x_axis_label='mean_freq', y_axis_label='mean_percussive', title="Densidade Mean_freq x mean_percussive")
fig_mean_freq_mean_percussive.circle(x=a_base.mean_freq, y=a_base.mean_percussive, color='#ffe6ee', legend='a')
fig_mean_freq_mean_percussive.circle(x=b_base.mean_freq, y=b_base.mean_percussive, color='#b5e5fb', legend='b')
fig_mean_freq_mean_percussive.circle(x=c_base.mean_freq, y=c_base.mean_percussive, color='#000000', legend='c')
fig_mean_freq_mean_percussive.circle(x=d_base.mean_freq, y=d_base.mean_percussive, color='#5f37ab', legend='d')
fig_mean_freq_mean_percussive.circle(x=h_base.mean_freq, y=h_base.mean_percussive, color='#298b95', legend='h')
fig_mean_freq_mean_percussive.circle(x=m_base.mean_freq, y=m_base.mean_percussive, color='#561c22', legend='m')
fig_mean_freq_mean_percussive.circle(x=n_base.mean_freq, y=n_base.mean_percussive, color='#610361', legend='n')
fig_mean_freq_mean_percussive.circle(x=x_base.mean_freq, y=x_base.mean_percussive, color='#d82aac', legend='x')
fig_mean_freq_mean_percussive.circle(x=si_base.mean_freq, y=si_base.mean_percussive, color='#bc8b79', legend='si')
fig_mean_freq_mean_percussive.circle(x=se_base.mean_freq, y=se_base.mean_percussive, color='#6c819e', legend='se')

show(fig_mean_freq_mean_percussive)

In [106]:

#Tempo X mean_percussive

fig_tempo_mean_percussive = figure(x_axis_label='tempo', y_axis_label='mean_percussive', title="Densidade tempo x mean_percussive")
fig_tempo_mean_percussive.circle(x=a_base.tempo, y=a_base.mean_percussive, color='#ffe6ee', legend='a')
fig_tempo_mean_percussive.circle(x=b_base.tempo, y=b_base.mean_percussive, color='#b5e5fb', legend='b')
fig_tempo_mean_percussive.circle(x=c_base.tempo, y=c_base.mean_percussive, color='#000000', legend='c')
fig_tempo_mean_percussive.circle(x=d_base.tempo, y=d_base.mean_percussive, color='#5f37ab', legend='d')
fig_tempo_mean_percussive.circle(x=h_base.tempo, y=h_base.mean_percussive, color='#298b95', legend='h')
fig_tempo_mean_percussive.circle(x=m_base.tempo, y=m_base.mean_percussive, color='#561c22', legend='m')
fig_tempo_mean_percussive.circle(x=n_base.tempo, y=n_base.mean_percussive, color='#610361', legend='n')
fig_tempo_mean_percussive.circle(x=x_base.tempo, y=x_base.mean_percussive, color='#d82aac', legend='x')
fig_tempo_mean_percussive.circle(x=si_base.tempo, y=si_base.mean_percussive, color='#bc8b79', legend='si')
fig_tempo_mean_percussive.circle(x=se_base.tempo, y=se_base.mean_percussive, color='#6c819e', legend='se')

show(fig_tempo_mean_percussive)

In [107]:
#freq_QT25 X mean_percussive

fig_freq_QT25_mean_percussive = figure(x_axis_label='freq_QT25', y_axis_label='mean_percussive', title="Densidade freq_QT25 x mean_percussive")
fig_freq_QT25_mean_percussive.circle(x=a_base.freq_QT25, y=a_base.mean_percussive, color='#ffe6ee', legend='a')
fig_freq_QT25_mean_percussive.circle(x=b_base.freq_QT25, y=b_base.mean_percussive, color='#b5e5fb', legend='b')
fig_freq_QT25_mean_percussive.circle(x=c_base.freq_QT25, y=c_base.mean_percussive, color='#000000', legend='c')
fig_freq_QT25_mean_percussive.circle(x=d_base.freq_QT25, y=d_base.mean_percussive, color='#5f37ab', legend='d')
fig_freq_QT25_mean_percussive.circle(x=h_base.freq_QT25, y=h_base.mean_percussive, color='#298b95', legend='h')
fig_freq_QT25_mean_percussive.circle(x=m_base.freq_QT25, y=m_base.mean_percussive, color='#561c22', legend='m')
fig_freq_QT25_mean_percussive.circle(x=n_base.freq_QT25, y=n_base.mean_percussive, color='#610361', legend='n')
fig_freq_QT25_mean_percussive.circle(x=x_base.freq_QT25, y=x_base.mean_percussive, color='#d82aac', legend='x')
fig_freq_QT25_mean_percussive.circle(x=si_base.freq_QT25, y=si_base.mean_percussive, color='#bc8b79', legend='si')
fig_freq_QT25_mean_percussive.circle(x=se_base.freq_QT25, y=se_base.mean_percussive, color='#6c819e', legend='se')

show(fig_freq_QT25_mean_percussive)

Gaussian Naive-Bayes

In [67]:
gnb = GaussianNB()
gnb.fit(train_base, Y)

GaussianNB(priors=None)

In [68]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if gnb.predict([test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)
        

210
582


Discriminante Linear

In [69]:
clf = LinearDiscriminantAnalysis()
clf.fit(train_base, Y)

/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [70]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if clf.predict([test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

221
571


KNN

In [102]:
tempo_mean_percussive_train_base = np.array(list(zip(freq_QT25, mean_percussive)))
norm_train = preprocessing.normalize(tempo_mean_percussive_train_base, norm='l1')
tempo_mean_percussive_test_base = np.array(list(zip(freq_QT25_test, mean_percussive_test)))
norm_test = preprocessing.normalize(tempo_mean_percussive_test_base, norm='l1')
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(norm_train, Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [103]:
false = 0
true = 0
for x in range(0, len(tempo_mean_percussive_test_base)):
    if neigh.predict([norm_test[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

85
707


Decision Tree

In [64]:
dtr = tree.DecisionTreeClassifier()
dtr = dtr.fit(tempo_mean_percussive_train_base, Y)

In [65]:
false = 0
true = 0
for x in range(0, len(test_base)):
    if dtr.predict([tempo_mean_percussive_test_base[x]])[0] == Y_test[x]:
        true += 1
    else:
        false += 1
print(true)
print(false)

157
635
